In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

import tldextract


def url2domain(url, exclude_subdomains=False):
    """Extract the domain from URL.
    """
    tld = tldextract.extract(url)
    a = []
    if tld.subdomain != '':
        if isinstance(exclude_subdomains, list):
            if tld.subdomain not in exclude_subdomains:
                a.append(tld.subdomain)
        elif not exclude_subdomains :
            a.append(tld.subdomain)
    a.append(tld.domain)
    if tld.suffix != '':
        a.append(tld.suffix)
    domain = '.'.join(a)
    return domain

NGRAMS = 2
FEATURE_LEN = 128
EPOCHS = 15
SAMPLES = 60000

df = pd.read_csv('./data/phishtank_2016.csv.bz2')
df.dropna(subset=['domain'], inplace=True)
df['domain'] = df.domain.apply(lambda c: url2domain(c, exclude_subdomains=['www']))
df

,ID,Phish URL,Submitted,Valid?,Online?,url,date,domain
0,3720185,http://facebooktayimagen.elpajarito.info/added...,by cleanmx,VALID PHISH,Offline,http://facebooktayimagen.elpajarito.info/,2015-12-31 08:50:00,facebooktayimagen.elpajarito.info
1,3720184,http://www.otsk.org/wp-content/www.ing.be/smar...,by cleanmx,VALID PHISH,Offline,http://www.otsk.org/wp-content/www.ing.be/smar...,2015-12-31 08:50:00,otsk.org
2,3720166,http://401771-deutschland-nutzung-angabe-nachw...,by cleanmx,VALID PHISH,Offline,http://401771-deutschland-nutzung-angabe-nachw...,2015-12-31 08:47:00,401771-deutschland-nutzung-angabe-nachweis.sic...
3,3720158,http://www.galatikohorio.com/dropboxfile/dropb...,by cleanmx,VALID PHISH,Offline,http://www.galatikohorio.com/dropboxfile/dropb...,2015-12-31 08:46:00,galatikohorio.com
4,3720152,http://shapecasting.com/images/35ba965121316b8...,by cleanmx,VALID PHISH,Offline,http://shapecasting.com/images/35ba965121316b8...,2015-12-31 08:45:00,shapecasting.com
5,3720151,http://shapecasting.com/images/35ba965121316b8...,by cleanmx,VALID PHISH,Offline,http://shapecasting.com/images/35ba965121316b8...,2015-12-31 08:45:00,shapecasting.com
6,3720150,http://femmebuzz.com/wp-includes/theme-compat/...,by GaryDee,VALID PHISH,Offline,http://femmebuzz.com/wp-includes/theme-compat/...,2015-12-31 08:45:00,femmebuzz.com
7,3720145,http://rigoletto.com.mx/follow/index.phpadded ...,by cleanmx,VALID PHISH,Offline,http://rigoletto.com.mx/follow/index.php,2015-12-31 08:44:00,rigoletto.com.mx
8,3720140,http://www.volivolifiji.com/dccde7bcf23f8a4dbb...,by cleanmx,VALID PHISH,Offline,http://www.volivolifiji.com/dccde7bcf23f8a4dbb...,2015-12-31 08:43:00,volivolifiji.com
9,3720132,http://www.motionfoundry.com.au/dec/cd3bb44cf8...,by cleanmx,VALID PHISH,Offline,http://www.motionfoundry.com.au/dec/cd3bb44cf8...,2015-12-31 08:42:00,motionfoundry.com.au


In [2]:
sdf = df.drop_duplicates('domain')
sdf

,ID,Phish URL,Submitted,Valid?,Online?,url,date,domain
0,3720185,http://facebooktayimagen.elpajarito.info/added...,by cleanmx,VALID PHISH,Offline,http://facebooktayimagen.elpajarito.info/,2015-12-31 08:50:00,facebooktayimagen.elpajarito.info
1,3720184,http://www.otsk.org/wp-content/www.ing.be/smar...,by cleanmx,VALID PHISH,Offline,http://www.otsk.org/wp-content/www.ing.be/smar...,2015-12-31 08:50:00,otsk.org
2,3720166,http://401771-deutschland-nutzung-angabe-nachw...,by cleanmx,VALID PHISH,Offline,http://401771-deutschland-nutzung-angabe-nachw...,2015-12-31 08:47:00,401771-deutschland-nutzung-angabe-nachweis.sic...
3,3720158,http://www.galatikohorio.com/dropboxfile/dropb...,by cleanmx,VALID PHISH,Offline,http://www.galatikohorio.com/dropboxfile/dropb...,2015-12-31 08:46:00,galatikohorio.com
4,3720152,http://shapecasting.com/images/35ba965121316b8...,by cleanmx,VALID PHISH,Offline,http://shapecasting.com/images/35ba965121316b8...,2015-12-31 08:45:00,shapecasting.com
6,3720150,http://femmebuzz.com/wp-includes/theme-compat/...,by GaryDee,VALID PHISH,Offline,http://femmebuzz.com/wp-includes/theme-compat/...,2015-12-31 08:45:00,femmebuzz.com
7,3720145,http://rigoletto.com.mx/follow/index.phpadded ...,by cleanmx,VALID PHISH,Offline,http://rigoletto.com.mx/follow/index.php,2015-12-31 08:44:00,rigoletto.com.mx
8,3720140,http://www.volivolifiji.com/dccde7bcf23f8a4dbb...,by cleanmx,VALID PHISH,Offline,http://www.volivolifiji.com/dccde7bcf23f8a4dbb...,2015-12-31 08:43:00,volivolifiji.com
9,3720132,http://www.motionfoundry.com.au/dec/cd3bb44cf8...,by cleanmx,VALID PHISH,Offline,http://www.motionfoundry.com.au/dec/cd3bb44cf8...,2015-12-31 08:42:00,motionfoundry.com.au
10,3720130,http://ebcv.org/administrator/ao/gucci.php?ip...,by cleanmx,VALID PHISH,Offline,http://ebcv.org/administrator/ao/gucci.php?ip...,2015-12-31 08:42:00,ebcv.org


In [3]:
try:
    sdf.groupby('target').agg({'domain': 'count'})
except:
    pass

In [4]:
adf = pd.read_csv('./data/top-1m.csv.zip', header=None)
adf.columns = ['rank', 'domain']
adf

,rank,domain
0,1,google.com
1,2,youtube.com
2,3,facebook.com
3,4,baidu.com
4,5,wikipedia.org
5,6,yahoo.com
6,7,google.co.in
7,8,reddit.com
8,9,qq.com
9,10,taobao.com


In [5]:
ldf = adf[['domain']].head(SAMPLES)
pdf = sdf[['domain']].sample(SAMPLES)

In [6]:
ldf['phishing'] = False
pdf['phishing'] = True
tdf = pd.concat([ldf, pdf])
tdf

,domain,phishing
0,google.com,False
1,youtube.com,False
2,facebook.com,False
3,baidu.com,False
4,wikipedia.org,False
5,yahoo.com,False
6,google.co.in,False
7,reddit.com,False
8,qq.com,False
9,taobao.com,False


## Preprocessing the input data

In [7]:
# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(tdf.domain)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(tdf.domain.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))

num_words = 1493
Max feature len = 62, Avg. feature len = 15


In [8]:
y = np.array(tdf.phishing.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## Train a LSTM model

In [9]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = FEATURE_LEN # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

/opt/venv/local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


96000 train sequences
24000 test sequences
Pad sequences (samples x time)
X_train shape: (96000, 128)
X_test shape: (24000, 128)
2 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (96000, 2)
y_test shape: (24000, 2)


In [10]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128*2, dropout=0.2, recurrent_dropout=0.2))

#model.add(Dense(num_classes, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 128, 32)           47776     
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               295936    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 344,226
Trainable params: 344,226
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=2)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 86400 samples, validate on 9600 samples
Epoch 1/15
 - 788s - loss: 0.4952 - acc: 0.7529 - val_loss: 0.4865 - val_acc: 0.7555
Epoch 2/15
 - 789s - loss: 0.4645 - acc: 0.7722 - val_loss: 0.4599 - val_acc: 0.7711
Epoch 3/15
 - 797s - loss: 0.4577 - acc: 0.7759 - val_loss: 0.4558 - val_acc: 0.7757
Epoch 4/15
 - 790s - loss: 0.4437 - acc: 0.7839 - val_loss: 0.4512 - val_acc: 0.7757
Epoch 5/15
 - 804s - loss: 0.4346 - acc: 0.7877 - val_loss: 0.4460 - val_acc: 0.7783
Epoch 6/15
 - 798s - loss: 0.4278 - acc: 0.7924 - val_loss: 0.4470 - val_acc: 0.7771
Epoch 7/15
 - 799s - loss: 0.4210 - acc: 0.7958 - val_loss: 0.4487 - val_acc: 0.7809
Epoch 8/15
 - 762s - loss: 0.4185 - acc: 0.7981 - val_loss: 0.4442 - val_acc: 0.7796
Epoch 9/15
 - 748s - loss: 0.4096 - acc: 0.8027 - val_loss: 0.4432 - val_acc: 0.7820
Epoch 10/15
 - 760s - loss: 0.4060 - acc: 0.8052 - val_loss: 0.4429 - val_acc: 0.7825
Epoch 11/15
 - 644s - loss: 0.4002 - acc: 0.8072 - val_loss: 0.4446 - val_acc: 0.7839
Epoch

## Confusion Matrix

In [12]:
y_pred = model.predict_classes(X_test, verbose=2)
p = model.predict_proba(X_test, verbose=2) # to predict probability
target_names = list(tdf.phishing.astype('category').cat.categories)

In [13]:
target_names = [str(t) for t in target_names]
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

             precision    recall  f1-score   support

      False       0.78      0.78      0.78     12000
       True       0.78      0.78      0.78     12000

avg / total       0.78      0.78      0.78     24000

[[9396 2604]
 [2623 9377]]


```
sigmoid

             precision    recall  f1-score   support

      False       0.88      0.76      0.81       200
       True       0.79      0.90      0.84       200

avg / total       0.83      0.82      0.82       400

[[151  49]
 [ 21 179]]
```

## Save model

In [14]:
model.save('./models/phish_cat_lstm_2016.h5')

In [15]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./models/phish_cat_vocab_2016.csv', index=False, encoding='utf-8')

In [16]:
p.sum(axis=1)

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)